# Import Librares

In [8]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
from sklearn import model_selection

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, r2_score

# Train XGBRegressor using splitData1

In [9]:
df = pd.read_csv("Dataset/splitData1_kfold_TrainAndValid.csv")
df.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,Stick_Length_is_missing,Thumb_is_missing,Pattern_Changer_is_missing,Grouser_Type_is_missing,Backhoe_Mounting_is_missing,Blade_Type_is_missing,Travel_Controls_is_missing,Differential_Type_is_missing,Steering_Controls_is_missing,kfold
0,1139248,57000.0,117657,77,121,3.0,1996,4640.0,2,1745,...,True,True,True,True,True,True,True,False,False,3
1,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,1,3716,...,True,True,True,True,True,True,True,True,True,0
2,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,3,4261,...,True,True,True,True,True,True,True,True,True,2
3,1139272,21500.0,1036251,36003,121,3.0,2008,302.0,2,788,...,False,False,False,False,True,True,True,True,True,1
4,1139290,28000.0,1058450,5162,121,3.0,2004,320.0,2,277,...,True,True,True,True,True,True,True,True,True,0


In [10]:
df.shape

(82540, 104)

# Using KFolds for cross validation

In [11]:
xgb_params = {
              'random_state': 1,
              'n_estimators': 20000,
              'objective': 'reg:squarederror',
              'n_jobs': -1,                 
              # Only test with these parameters
              # 'gpu_id': 0,
              # 'predictor': 'gpu_predictor',
              # 'tree_method': 'gpu_hist',
              # Only test with these parameters
              }

In [ ]:
folds=5
total_mean_rmse = 0
total_mean_mae = 0

for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)

    ytrain = xtrain.SalePrice
    yvalid = xvalid.SalePrice

    model = XGBRegressor(**xgb_params)
    
    # fit, validate and score       
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict(xvalid)
    
    # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html
    # If True returns MSE value, if False returns RMSE value.
    fold_rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    fold_mae = mean_absolute_error(yvalid, preds_valid)

    print(fold, fold_rmse)
    print(fold, fold_mae)
   
    # calculate mean RMSE and MAE
    total_mean_rmse += fold_rmse / folds
    total_mean_mae += fold_mae / folds
  
print(f"Total Mean RMSE: {total_mean_rmse}")
print(f"Total Mean MAE: {total_mean_mae}")

[0]	validation_0-rmse:27320.50781
[1000]	validation_0-rmse:13.95063
[2000]	validation_0-rmse:13.93765
